In [1]:
look_back = 30
file_modello = "LSTM.keras"
prev_true = 0.5

In [2]:
def visualizza_grafico(ticker):
    import plotly.graph_objs as go
    fig = fx.grafico_base(ticker)
    tg = go.Scatter(
        x = ticker[ticker['Target'] == True].index,
        y = ticker[ticker['Target'] == True]['Close'],
        mode = 'markers',
        marker = dict(
            size = 30,
            color = 'rgba(100, 149, 250, .8)'
        ),
        name = 'target'
    )
    pred = go.Scatter(
        x = ticker[ticker['Pred'] == True].index,
        y = ticker[ticker['Pred'] == True]['Close'],
        mode = 'markers',
        marker = dict(
            size = 15,
            color = 'rgba(255, 200, 15, .8)'
        ),
        name = 'pred'
    )
    fig.add_trace(tg, row=1, col=1)
    fig.add_trace(pred, row=1, col=1)
    pyo.plot(fig, filename="test_modello.html", auto_open=True)

In [3]:
import pandas as pd
import yfinance as yf
import funzioni as fx
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
import plotly.offline as pyo

pd.set_option("display.max_columns", None)
elenco_usin = pd.read_parquet("Tickers_De_Giro.parquet")
elenco_usin = elenco_usin.loc[elenco_usin["Categoria"] != "D", ["USIN", "Ticker"]]
elenco_usin = elenco_usin.sample(frac=1)
model = load_model(file_modello)
usin = elenco_usin.iloc[0]
print(f"\rScarico dati ticker {usin['Ticker']}                                             ", end=" ", flush=True)
ticker = yf.download(usin['USIN'], start='2015-01-01', end='2023-07-31', progress=False)
print(f"\rCalcolo indicatori ticker {usin['Ticker']}                                             ", end=" ", flush=True)
ticker = fx.crea_indicatori(ticker)
features, target = fx.features_e_target(ticker)
ticker["Target"] = target
X, Y = fx.converti_in_XY(features, target, look_back)
Y = np.where(Y, 1, 0)

print(f"\rScalamento dati ticker {usin['Ticker']}                                             ", end=" ", flush=True)
scaler = StandardScaler()
n_samples, n_timesteps, n_features = X.shape
X_reshape = X.reshape((n_samples, n_timesteps * n_features))
X_scaled = scaler.fit_transform(X_reshape)
X_scaled_reshape = X_scaled.reshape((-1, n_timesteps, n_features))

print(f"\rPrevisione {usin['Ticker']}                                             \n", end=" ", flush=True)
pred = model.predict(X_scaled_reshape)

ticker = ticker.iloc[n_timesteps:]

Previsione FCX                                                             
61/61 [==============================] - 3s 18ms/step


In [4]:
ticker["Pred"] = pred > prev_true
ticker["prob_True"] = pred
cf = pd.DataFrame(confusion_matrix(ticker["Target"], ticker["Pred"]), index=[False, True], columns=["Prev.False", "Prev.True"])
cf["Totale"] = cf["Prev.False"] + cf["Prev.True"]
cf.loc["Totale"] = cf.sum()


In [5]:
pos_aperta = False
tp = sl = n_azioni = prezzo_acquisto = prezzo_tot = 0
bilancio = 1000

posizioni = pd.DataFrame(columns=["Bilancio", "Direzione", "Prezzo_un", "n_azioni", "Prezzo_tot", "TP", "SL", "Open", "High", "Low", "Esito"], index=ticker.index)

for idx, row in ticker.iterrows():
    if bilancio <= 0:
        break
    print(f"\r{idx} Bilancio = {bilancio}                ", end=" ", flush=True)
    if pos_aperta == False:
        if row["Pred"] and bilancio > 0:
            #COMPRA
            prezzo_acquisto = row["Open"]
            n_azioni = bilancio // prezzo_acquisto
            prezzo_tot = n_azioni * prezzo_acquisto  
            bilancio -= prezzo_tot          
            tp = prezzo_acquisto * 1.2
            sl = prezzo_acquisto * 0.95
            pos_aperta = True
            posizioni.loc[idx] = {"Bilancio": bilancio, "Direzione": "COMPRA", "Prezzo_un": prezzo_acquisto, "n_azioni": n_azioni, "Prezzo_tot": -prezzo_tot, "TP": tp, "SL": sl, "Open": row["Open"], "High": row["High"], "Low": row["Low"],}
    else:
        if row["High"] >= tp:
            prezzo_tot = tp * n_azioni
            bilancio += prezzo_tot
            pos_aperta = False
            posizioni.loc[idx] = {"Bilancio": bilancio, "Direzione": "VENDI", "Prezzo_un": tp, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "Vincita", "Open": row["Open"], "High": row["High"], "Low": row["Low"]}
        elif row["Low"] <= sl:
            prezzo_tot = sl * n_azioni
            bilancio += prezzo_tot
            pos_aperta = False
            posizioni.loc[idx] = {"Bilancio": bilancio, "Direzione": "VENDI", "Prezzo_un": sl, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "Perdita", "Open": row["Open"], "High": row["High"], "Low": row["Low"]}
    
posizioni.to_excel("posizioni.xlsx")
ticker.to_excel("ticker.xlsx")
print("\nfine")


2023-03-06 00:00:00 Bilancio = 816.062998819351                                      
fine


In [6]:
posizioni.loc[posizioni["Direzione"] == "VENDI", :].tail()

,Bilancio,Direzione,Prezzo_un,n_azioni,Prezzo_tot,TP,SL,Open,High,Low,Esito
Date,,,,,,,,,,,
2017-02-07,950.679998,VENDI,15.618001,60.0,937.08003,NaN,NaN,16.01,16.08,15.41,Perdita
2017-08-11,903.359997,VENDI,13.832,65.0,899.080026,NaN,NaN,14.0,14.1,13.82,Perdita
2018-02-05,858.407998,VENDI,17.7935,48.0,854.087979,NaN,NaN,18.139999,18.65,17.360001,Perdita
2018-03-23,816.062999,VENDI,17.879,45.0,804.554987,NaN,NaN,18.139999,18.23,17.51,Perdita


In [7]:
cf

,Prev.False,Prev.True,Totale
False,1842,19,1861
True,67,0,67
Totale,1909,19,1928


In [8]:
#visualizza_grafico(ticker)